# Register ADK Agent On Vertex AI Agent Engine With Gemini Enterprise

Previously Gemini Enterprise was called Agentspace.

In [5]:
import subprocess
import json
import requests
import dotenv
from vertexai import agent_engines

In [2]:
PROJECT_ID = subprocess.run(['gcloud', 'config', 'get-value', 'project'], capture_output=True, text=True, check=True).stdout.strip()
PROJECT_NUMBER = subprocess.run(['gcloud', 'projects', 'describe', PROJECT_ID, '--format=value(projectNumber)'], capture_output=True, text=True, check=True).stdout.strip()

print(f"\n{'='*50}\nGoogle Cloud Project Information\n{'='*50}\nPROJECT_ID     = {PROJECT_ID}\nPROJECT_NUMBER = {PROJECT_NUMBER}\n{'='*50}\n")


Google Cloud Project Information
PROJECT_ID     = statmike-mlops-349915
PROJECT_NUMBER = 1026793852137



## Get ADK Agent on Vertex AI Agent Engine

In [3]:
agent_folder = './agent_convo_api'
agent_deploy_folder = agent_folder + '/deploy'

In [6]:
config = dotenv.dotenv_values(dotenv_path = f'{agent_deploy_folder}/.env')
current_deployment = config.get('ADK_DEPLOY_RESOURCE_ID')
current_deployment

'projects/1026793852137/locations/us-central1/reasoningEngines/6919838002059411456'

In [7]:
try:
    remote_app = agent_engines.get(resource_name = current_deployment)
except Exception as e:
    remote_app = None

In [8]:
remote_app.resource_name

'projects/1026793852137/locations/us-central1/reasoningEngines/6919838002059411456'

In [9]:
remote_app.display_name

'agent_convo_api'

In [12]:
remote_app.to_dict()['description']

'An agent that can answer questions about BigQuery data using the Conversational Analytics API.'

In [15]:
LOCATION = "global"  # Or your specific location, e.g., "us-central1"
AGENT_DISPLAY_NAME = remote_app.display_name
AGENT_DESCRIPTION = remote_app.to_dict()['description']
APP_ID = "enterprise-search-17381091_1738109128316"

In [18]:
response = requests.post(
    url=f"https://discoveryengine.googleapis.com/v1alpha/projects/{PROJECT_NUMBER}/locations/{LOCATION}/collections/default_collection/engines/{APP_ID}/assistants/default_assistant/agents",
    headers={
        "Authorization": f"Bearer {subprocess.check_output(['gcloud', 'auth', 'print-access-token'], text=True).strip()}",
        "Content-Type": "application/json",
        "X-Goog-User-Project": PROJECT_NUMBER
    },
    json={
        "displayName": AGENT_DISPLAY_NAME,
        "description": AGENT_DESCRIPTION,
        "adk_agent_definition": {
            "tool_settings": {
                "tool_description": AGENT_DESCRIPTION
            },
            "provisioned_reasoning_engine": {
                "reasoning_engine": remote_app.resource_name
            }
        },
    }
)

In [19]:
print(f"Status Code: {response.status_code}")
if response.headers.get('Content-Type') == 'application/json':
    # The .json() method will raise an error if the response is not valid JSON
    print(response.json())
else:
    print(response.text)

Status Code: 200
{
  "name": "projects/1026793852137/locations/global/collections/default_collection/engines/enterprise-search-17381091_1738109128316/assistants/default_assistant/agents/18337555184602126877",
  "displayName": "agent_convo_api",
  "description": "An agent that can answer questions about BigQuery data using the Conversational Analytics API.",
  "createTime": "2025-10-15T16:55:46.862102849Z",
  "adkAgentDefinition": {
    "toolSettings": {
      "toolDescription": "An agent that can answer questions about BigQuery data using the Conversational Analytics API."
    },
    "provisionedReasoningEngine": {
      "reasoningEngine": "projects/1026793852137/locations/us-central1/reasoningEngines/6919838002059411456"
    }
  },
  "state": "ENABLED"
}

